## Web Scraping

This lecture provides foundational knowledge and examples of scraping/pulling data from live websites.

---

### Set Up
1.	Ensure you have a stable internet connection
2.  Ensure the chromedriver app is in the same folder as this notebook. If you pulled correctly from github, this shouldn't be an issue!


### Needed Packages
1.	pandas
2.  selenium
3.  bs4
4.  requests
5.  random
6.  time
7.  warnings

### Nice Chrome Extensions to Have
1. <a href='https://chrome.google.com/webstore/detail/xpath-finder/ihnknokegkbpmofmafnkoadfjkhlogph?hl=en'>xpath-finder</a>
2. <a href='https://chrome.google.com/webstore/detail/relative-xpath-helper/eanaofphbanknlngejejepmfomkjaiic?hl=en'>relative-xpath-helper</a>
---

# Table of Contents

### The Basics
#### <a href='#1'>What is web scraping and why do you need it?</a>
#### <a href='#2'>Needed Packages</a>
#### <a href='#3'>Scraping ESPN</a>
#### <a href='#4'>A note on anonymous web scraping and browsing</a>

### Advanced Methods
#### <a href='#5'>Creating an email address through python only</a>
#### <a href='#6'>Download Consumer Spending Data</a>
#### <a href='#7'>Scrape current US National Debt</a>

### Additional Use cases
#### <a href='#8'>Scrape images and other files</a>
#### <a href='#9'>Scraping function to download files of any type from a website</a>
#### <a href='#10'>Scrape funny coffee pictures</a>
#### <a href='#11'>Scrape Bloomberg sitemap (XML) for current political news</a>
#### <a href='#12'>Web crawl Twitter account</a>
#### <a href='#17'>Making User Entry Hidden (Great for Passwords)</a>
#### <a href='#18'>Making Web Browser Wait with Time.Sleep()</a>

### Practice
#### <a href='#13'>Exercise Set 1</a>
#### <a href='#14'>Exercise Set 2</a>
#### <a href='#15'>Exercise Set 3</a>
#### <a href='#16'>Exercise Set 4</a>

<a id='1'></a>
## What is web scraping and why do you need it?
Web scraping is a catch all term for gathering information directly from web pages on the internet.  Unlike structured APIs and Databases, web scraping will require you to get creative in how you approach collecting the data as no two projects will be the same.  As an important addition to webscraping, this lecture will also cover basic use of the selenium package, which will enable you to traverse and interact with the vast majority of modern webpages, ranging from obscure pages all the way to Facebook.

In general, web scraping should be your last resort when looking to automate a data gathering exercise, as websites are prone to change over time, many web hosts do not appreciate web scraping, and it's generally much slower than just working with prebuilt data sources.  

<a id='2'></a>
## Needed Packages

In [1]:
import pandas as pd
import warnings
import selenium
from bs4 import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import getpass
from random import randint
import requests
warnings.filterwarnings('ignore')

In [2]:
# If using a Mac, need to specify path that contains chromedriver
driver_path = "/Users/Matthew/Documents/Consulting/Lectures/Webscraping/chromedriver"

<a id='3'></a>
## Scraping ESPN

In [3]:
##define website url
url = 'https://www.espn.com/nba/boxscore/_/gameId/401360104'

In [4]:
##Open browser

# If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver')

# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path)
browser.get(url)

In [5]:
##Close browser
browser.close()

In [6]:
##Open browser without actually seeing it, also known as 'headless' browsing
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')

#If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver.exe', chrome_options=chrome_options)

#If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

browser.get(url)

In [7]:
##Grab the raw webpage
innerHTML = browser.execute_script("return document.body.innerHTML")

In [8]:
##Parse raw webpage and close the browser (Don't want to gunk up your RAM!!)
soup = BeautifulSoup(innerHTML,"html.parser")
browser.close()

In [9]:
##Find all tables on page
tables = soup.findAll("table")

In [10]:
##Grab Stats for all players on both teams & convert to dataframe
df = []
for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        cells = [ele.text.strip() for ele in cells]
        if len(cells) == 15 and cells[0] != '' and cells[0] != 'TEAM':
            df.append(cells[0:3])
df = pd.DataFrame(df, columns = ['Player','Minutes Played','FG'])
df.head()

,Player,Minutes Played,FG
0,J. GrantJ. GrantSF,33,5-16
1,S. BeyS. BeySF,23,4-12
2,I. StewartI. StewartC,27,2-6
3,C. JosephC. JosephPG,25,3-6
4,C. CunninghamC. CunninghamPG,28,3-13


<a id='4'></a>
## A note on anonymous web scraping and browsing
When web scraping, many websites will blacklist your IP address in an effort to prevent you from abusing their sites.  If you are at a job and your employer is needing the data, an IP ban is not something you want ruining your day.  A common work around is to simply use a proxy server so that the website doesn't know your actual IP address.  The below will give the framework to put a proxy server between you and the internet calls you are making with python.  

In [15]:
##Proxy server list
## 'https://proxydig.com/free-proxy-list/'

In [16]:
##Connect to a website using a proxy server
PROXY = '209.165.163.187:3128'  ##Note that if you cannot connect to a webpage, try using a different proxy server from the site above
url = 'https://whatismyipaddress.com/'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver.exe',chrome_options=chrome_options)

#If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

browser.get(url)

##NOTE: Using proxy servers will avoid most common website black listing BUT, does not substitute a VPN for security NOR
##should you attempt to use proxy servers for nefarious activity

WebDriverException: Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=101.0.4951.54)


While proxy servers enable you to avoid blacklisting, they leave you exposed from a security standpoint.  There are two common methods to add a layer of security.  The first is to always work with a VPN on before doing any web scraping, this will ensure even if someone tries to trace back to the original IP, only the VPN's will appear.  Alternatively, Selenium allows you to use the Tor browser to make internet calls.  This method is generally extremely slow, and unnecessary, but a fun exercise if you have a couple hours to spare! 

<a id='13'></a>
## -------------PRACTICE-------------

1. Create a function that will open and return a headless web browser, think of variables that may be helpful to send the function.  

2. Go to: https://www.macrotrends.net/1333/historical-gold-prices-100-year-chart and scrape the table titled 'Gold Prices - Historical Annual Data'

3.  Put the data from question 2 in a pandas dataframe.

<a id='5'></a>
## Creating an email address through python only


In [ ]:
##Open webpage
url = 'https://mail.tutanota.com/login'

# If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver.exe',chrome_options=chrome_options)

# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

browser.get(url)

In [ ]:
##Select 'More' so we can create an account
more_button = browser.find_element_by_xpath('//*[@id="login-view"]/div[2]/div/div[3]/div/button')
more_button.click()

In [ ]:
##Select 'Sign Up' so we can create an account
sign_up_button = browser.find_element_by_xpath('//*[@id="login-view"]/div[2]/div/div[4]/div/div/div/button[1]/div')
sign_up_button.click()

In [ ]:
##Select the Free option
free_button = browser.find_element_by_xpath('//*[@id="upgrade-account-dialog"]/div[2]/div[1]/div[1]/div[5]/button/div')
free_button.click()

In [ ]:
##Agree to terms
term_1 = browser.find_element_by_xpath('//*[@id="modal"]/div[2]/div/div/div/div[2]/div[1]/div/input')
term_1.click()
term_2 = browser.find_element_by_xpath('//*[@id="modal"]/div[2]/div/div/div/div[2]/div[2]/div/input')
term_2.click()
ok_button = browser.find_element_by_xpath('//*[@id="modal"]/div[2]/div/div/div/div[3]/button[2]/div')
ok_button.click()

In [ ]:
##Add Account Info
email_add = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[1]/div/div/div/div[1]/input')
email_add.click()
email_add.send_keys('test_user_abc_124') ###You will need to put in a new username!

password = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[2]/div[1]/div/div/div/div[1]/input[4]')
password.click()
password.send_keys('Sample_password!')

sec_password = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[2]/div[3]/div/div/div/div/input')
sec_password.click()
sec_password.send_keys('Sample_password!')

In [ ]:
##Agree to terms 2
term_1 = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[3]/div/input')
term_1.click()
term_2 = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[4]/div/input')
term_2.click()
ok_button = browser.find_element_by_xpath('//*[@id="signup-account-dialog"]/div/div[5]/button/div')
ok_button.click()

In [ ]:
##Continue
ok_button = browser.find_element_by_xpath('//*[@id="wizardDialogContent"]/div[4]/div/button/div')
ok_button.click()

In [ ]:
##Type in Password
password_input = browser.find_element_by_xpath('//*[@id="login-view"]/div[2]/div/div[1]/form/div[2]/div/div/div/div/div/input')
password_input.click()
password_input.send_keys('Sample_password!')

In [ ]:
##Login
login_button = browser.find_element_by_xpath('//*[@id="login-view"]/div[2]/div/div[1]/form/div[4]/button/div')
login_button.click()
browser.close()

<a id='14'></a>
## -------------PRACTICE-------------

1. Login to your new email account.

2. Click the button to send an email.

3. Send yourself an email.

<a id='6'></a>
## Download Consumer Spending Data

In [ ]:
##Open Webpage and make full screen
url = 'https://www.bea.gov/'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

# If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver.exe',chrome_options=chrome_options)

# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

browser.get(url)

In [ ]:
##Navigate to Data section
data = browser.find_element_by_link_text('Data')
data.click()

##Note that if you don't leave the browser in max view, this won't work!

In [ ]:
##Navigate to Data by Topic section
data = browser.find_element_by_partial_link_text('Data by Topic')
data.click()

In [ ]:
##Navigate to Consumer Spending Section
data = browser.find_element_by_link_text('Consumer Spending')
data.click()

In [ ]:
##Navigate to next Consumer Spending Section
data = browser.find_element_by_xpath('//*[@id="test"]/div[2]/article/div/div/div/ul/li[1]/a')
data.click()

In [ ]:
##Navigate to Interactive Data
data = browser.find_element_by_link_text('Interactive Data')
data.click()

In [ ]:
##Navigate to Summary Tables
data = browser.find_element_by_link_text('Summary Tables')
data.click()

In [ ]:
##Navigate to Person Income and Outlays
data = browser.find_element_by_partial_link_text('PERSONAL INCOME AND OUTLAYS')
data.click()

In [ ]:
##Navigate to table 2.2A
data = browser.find_element_by_partial_link_text('2.2A')
data.click()

In [ ]:
##Download Data
data = browser.find_element_by_xpath('//*[@id="showDownload"]')
data.click()

In [ ]:
##Select Download Format
data = browser.find_element_by_xpath('//*[@id="download_wraper"]/div/a[2]')
data.click()
browser.close()

<a id='15'></a>
## -------------PRACTICE-------------

1. Navigate to a different data source.

2. Download a different data set than what was downloaded above.

3. Read the new data set into a pandas dataframe.

<a id='7'></a>
## Scrape current US National Debt

In [ ]:
##Open Webpage and make full screen
url = 'https://www.usdebtclock.org/'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')

# If using Mac
#browser = webdriver.Chrome(executable_path = 'chromedriver.exe',chrome_options=chrome_options)

# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

browser.get(url)

In [ ]:
##Grab the raw webpage
innerHTML = browser.execute_script("return document.body.innerHTML")

In [ ]:
##Parse raw webpage and close the browser (Don't want to gunk up your RAM!!)
soup = BeautifulSoup(innerHTML,"html.parser")
browser.close()

In [ ]:
##Scrape the First number showing the total debt
divs = soup.findAll("div")
count = 1
for div in divs:
    for row in div.findAll("span"):
        if count == 1:
            print(row.text.strip())
            count+=1
        else:
            break

<a id='8'></a>
## Scrape images and other files
Let's see how we can automatically find and download files linked at any website.

The data you need for your projects might not always be raw data, but in the form of files (images, .txt files etc)

In [ ]:
# Images are displayed with the <img> tag in HTML

# open connection and create new soup

raw = requests.get('https://www.eatthis.com/unhealthiest-foods-on-the-planet/').content
soup = BeautifulSoup(raw,features='html.parser')

print(soup.find('img')) 
# as we can see below the image urls 
# are stored in the src attribute inside the img tag

In [ ]:
# Parse all url to the images
img_urls = list()
for img in soup.find_all('img'):
    img_url = img.get('src') 
    if '.jpeg' in img_url or '.jpg' in img_url:
        print(img_url)
        img_urls.append(img_url)

In [ ]:
## Let's look at what our current file directory looks like

%ls

In [ ]:
# To download and save files with Python we can use 
# the shutil library which is a file operations library
'''
The shutil module offers a number of high-level operations on files and 
collections of files. In particular, functions are provided which support 
file copying and removal.
'''

import shutil

for idx, img_url in enumerate(img_urls): 
    #enumarte to create a file integer name for every image
    
    # make a request to the image URL
    img_source = requests.get(img_url, stream=True) 
    # we set stream = True to download/ 
    # stream the content of the data
    
    with open('Spam/img'+str(idx)+'.jpg', 'wb') as file: 
        # open file connection, create file and write to it
        shutil.copyfileobj(img_source.raw, file) 
        # save the raw file object

    del img_source # to remove the file from memory

In [ ]:
## Let's see if the file has been saved
%ls

<a id='9'></a>
## Scraping function to download files of any type from a website
Below is a function that takes in a website and a specific file type to download X of them from the website.

In [ ]:
# Extended scraping function of any file format
import os # To interact with operating system and format file name
import shutil # To copy file object from python to disk
import requests
import bs4 as bs

# source: https://github.com/scetx/datax/blob/master/03-data-x-data-handling/m310-tools-webscraping/nb-m310-tools-webscraping.ipynb
def py_file_scraper(url, html_tag='img', source_tag='src', file_type='.jpg',max=-1):
    
    '''
    Function that scrapes a website for certain file formats.
    The files will be placed in a folder called "files" 
    in the working directory.
    
    url = the url we want to scrape from
    html_tag = the file tag (usually img for images or 
    a for file links)
    
    source_tag = the source tag for the file url 
    (usually src for images or href for files)
    
    file_type = .png, .jpg, .pdf, .csv, .xls etc.
    
    max = integer (max number of files to scrape, 
    if = -1 it will scrape all files)
    '''
    
    # make a directory called 'files' 
    # for the files if it does not exist
    if not os.path.exists('files/'):
        os.makedirs('files/')
    print('Loading content from the url...')
    source = requests.get(url).content
    print('Creating content soup...')
    soup = bs.BeautifulSoup(source,'html.parser')
    
    i=0
    print('Finding tag:%s...'%html_tag)
    for n, link in enumerate(soup.find_all(html_tag)):
        file_url=link.get(source_tag)
        print ('\n',n+1,'. File url',file_url)
        
        
        if 'http' in file_url: # check that it is a valid link
            print('It is a valid url..')
            
            
            if file_type in file_url: #only check for specific 
                # file type
                
                print('%s FILE TYPE FOUND IN THE URL...'%file_type)
                file_name = os.path.splitext(os.path.basename(file_url))[0] + file_type 
                #extract file name from url

                file_source = requests.get(file_url, stream = True)
             
                # open new stream connection

                with open('./files/'+file_name, 'wb') as file: 
                    # open file connection, create file and 
                    # write to it
                    
                    shutil.copyfileobj(file_source.raw, file) 
                    # save the raw file object
                    
                    print('DOWNLOADED:',file_name)
                    
                    i+=1
                    
                del file_source # delete from memory
            else:
                print('%s file type NOT found in url:'%file_type)
                print('EXCLUDED:',file_url) 
                # urls not downloaded from
                
        if i == max:
            print('Max reached')
            break
            

    print('Done!')

<a id='10'></a>
## Scrape funny coffee pictures

In [ ]:
py_file_scraper('https://goldcoffee.com/dark/') 
# scrape coffee

<a id='11'></a>
## Scrape Bloomberg sitemap (XML) for current political news

In [ ]:
# XML documents - site maps, all the urls. just between tags
# XML human and machine readable.
# Newest links: all the links for FIND SITE MAP!
# News websites will have sitemaps for politics, bot constantly
# tracking news track the sitemaps

# Before scraping a website look at robots.txt file
bs.BeautifulSoup(requests.get('https://www.bloomberg.com/robots.txt').content,'lxml')

In [ ]:
source = requests.get('https://www.bloomberg.com/feeds/bpol/sitemap_news.xml').content
soup = bs.BeautifulSoup(source,'xml') # Note parser 'xml'

In [ ]:
print(soup.prettify())

In [ ]:
# Find political news headlines
for news in soup.find_all({'news'}):
    print(news.title.text)
    print(news.publication_date.text)
    #print(news.keywords.text)
    print('\n')

<a id='12'></a>
## Web crawl Twitter account

In [ ]:
# Helper function to maintain the urls and the number of times they appear

url_dict = dict()

def add_to_dict(url_d, key):
    if key in url_d:
        url_d[key] = url_d[key] + 1
    else:
        url_d[key] = 1

In [ ]:
# Recursive function which extracts links from the given url upto a given 'depth'.

def get_urls(url, depth):
    if depth == 0:
        return
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for link in soup.find_all('a'):
        if link.has_attr('href') and "https://" in link['href']:
#             print(link['href'])
            add_to_dict(url_dict, link['href'])
            get_urls(link['href'], depth - 1)

In [ ]:
# Iterative function which extracts links from the given url upto a given 'depth'.

def get_urls_iterative(url, depth):
    urls = [url]
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        for link in soup.find_all('a'):
            if link.has_attr('href') and "https://" in link['href']:
                add_to_dict(url_dict, link['href'])
                urls.append(link['href'])
        if len(urls) > depth:
            break

In [ ]:
get_urls("https://twitter.com/GolfWorld", 2)
for key in url_dict:
    print(str(key) + "  ----   " + str(url_dict[key]))

<a id='17'></a>
## Making User Entry Hidden (Great for Passwords)

getpass() prompts the user for a password without echoing. The getpass module provides a secure way to handle the password prompts where programs interact with the users via the terminal.

In [ ]:
# Need to import this package to have the hidden password functionality
import getpass
 
# Set password as variable p, put your question as a string for the function argument
# Notice that the password is not shown when entered
p = getpass.getpass("What is your password? ")

In [ ]:
# The password is stored within the memory of the program though
print('Password entered:', p)

<a id='18'></a>
## Making Web Browser Wait with Time.Sleep()

Python time method sleep() suspends execution for the given number of seconds. The argument may be a floating point number to indicate a more precise sleep time.  The actual suspension time may be less than that requested because any caught signal will terminate the sleep() following execution of that signal's catching routine.

source: https://www.tutorialspoint.com/python/time_sleep.htm

This function is very important to use in automation in conjunction with chromedriver.  Some elements on a web page may take a few seconds for your computer to load and trying to access them before that can result in an error.  This function tells Python to wait a certain number of seconds before it tries to run the next command.

In [ ]:
# Set time in seconds that we want to wait as 8 and store in variable t
t = 8

print('We have set the the variable t')

# Simply call time.sleep() with the seconds to wait as the function's argument
# Here we are waiting 8 seconds
time.sleep(t)

print('It has been 8 seconds')

# Or simply just use an integer as the argument
# Here we are waiting 5 seconds
time.sleep(5)
print('It has been 5 more seconds')

<a id='16'></a>
## -------------PRACTICE-------------

1. Go to a new site and download at least 2 images. 

2. Scrape a different twitter account.

3. Get the latest bloomberg news topic.

## Further Readings:

Thinks to keep in mind as you scrape data: https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01